In [201]:
import pandas as pd
import numpy as np

In [202]:
listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

In [203]:
listing_details.shape[0]

32461

In [204]:
listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27246 entries, 0 to 27245
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27246 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27246 non-null  object 
 2   Xã/Phường/Thị trấn                    26947 non-null  object 
 3   Đường phố                             20229 non-null  object 
 4   Chi tiết                              27246 non-null  object 
 5   Nguồn thông tin                       27246 non-null  object 
 6   Tình trạng giao dịch                  27246 non-null  object 
 7   Thời điểm giao dịch/rao bán           27236 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24742 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27246 non-null  object 
 11  Số tầng công tr

In [205]:
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27246 entries, 0 to 27245
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27246 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27246 non-null  object 
 2   Xã/Phường/Thị trấn                    26947 non-null  object 
 3   Đường phố                             20229 non-null  object 
 4   Chi tiết                              27246 non-null  object 
 5   url                                   27246 non-null  object 
 6   Tình trạng giao dịch                  27246 non-null  object 
 7   Thời điểm giao dịch/rao bán           27236 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24742 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27246 non-null  object 
 11  Số tầng công tr

In [206]:
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27248 entries, 0 to 27247
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27248 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27248 non-null  object 
 2   Xã/Phường/Thị trấn                    26949 non-null  object 
 3   Đường phố                             20231 non-null  object 
 4   Chi tiết                              27248 non-null  object 
 5   url                                   27248 non-null  object 
 6   Tình trạng giao dịch                  27248 non-null  object 
 7   Thời điểm giao dịch/rao bán           27238 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24744 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27248 non-null  object 
 11  Số tầng công tr

## Columns to fix: 
Xã/Phường/Thị trấn, Đường phố, Giá rao bán/giao dịch, Số tầng công trình, Tổng diện tích sàn, Đơn giá xây dựng, Chất lượng còn lại, Diện tích đất (m2), Kích thước mặt tiền (m), Kích thước chiều dài (m), Số mặt tiền tiếp giáp, Hình dạng, Độ rộng ngõ/ngách nhỏ nhất (m), Khoảng cách tới trục đường chính (m)

In [207]:
import numpy as np

def correct_phuong(row):
    xaphuong = row['Xã/Phường/Thị trấn']
    short_add = row['short_address']
    if type(xaphuong) == float or type(short_add) == float:
        return 0
    if xaphuong in short_add:
        return 1
    return 0

cleaned['correct_phuong'] = cleaned.apply(correct_phuong, axis=1)
incorrect_phuong = cleaned[cleaned['correct_phuong'] == 0]

In [208]:
pd.reset_option('display.max_colwidth', None)
pd.reset_option('display.max_rows', None)

incorrect_phuong[['Xã/Phường/Thị trấn', 'short_address']]

,Xã/Phường/Thị trấn,short_address
179,NaN,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...
308,NaN,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...
365,Phường Yên Nghĩa,"Yên Nghĩa, Hà Đông, Hà Nội"
373,NaN,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội"
402,Phường Mỹ Đình 2,"Mỹ Đình 2, Nam Từ Liêm, Hà Nội"
...,...,...
27154,NaN,"DH507, An Linh, Phú Giáo, Bình Dương"
27162,Xã Liệp Nghĩa,"Liệp Nghĩa, Quốc Oai, Hà Nội"
27170,Xã Phú Mãn,"Phú Mãn, Quốc Oai, Hà Nội"
27173,Thị Trấn Trâu Quỳ,"Trâu Quỳ, Gia Lâm, Hà Nội"


- 176: Không điền nổi
- 703: Phường Ba: Bạch Mai, Hai Bà Trưng, Hà Nội ???

__Correct values__: 306, 397, 470, 547, 833, 964, 966, 1103, 1606, 1657, 1663, 1694, 2184, 2414, 2417, 2551, 

__NaN values__
- 305: Nguyễn Viết Xuân Hòa Minh Liên Chiểu
- 368: Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội
- 474: Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội
- 564: Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh
- 619: Thạch Cầu, Long Biên, Long Biên, Hà Nội
- 706: Liên Mạc, Thượng Cát, Bắc Từ Liêm, Hà Nội
- 772: Trịnh Văn Bô, Xuân Phương, Nam Từ Liêm, Hà Nội
- 827: Đông Tác, Kim Liên, Đống Đa, Hà Nội
- 828: Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh
- 1042: Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh
- 1105: Nguyễn Trãi, Nguyễn Trãi, Thượng Đình, Thanh Xuân, Hà Nội
- 1169: Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh
- 1283: Trịnh Như Khuê, Bình Chánh, Bình Chánh, Hồ Chí Minh
- 1727: Nguyễn Văn Bứa, Xuân Thới Sơn, Hóc Môn, Hồ Chí Minh
- 1908: Xuân Đỉnh, Xuân Đỉnh, Bắc Từ Liêm, Hà Nội
- 1994: Cống Quỳnh, Nguyễn Cư Trinh, Quận 1, Hồ Chí Minh
- 2052: Vạn Kiếp, 3, Bình Thạnh, Hồ Chí Minh
- 2111: 27, 16, Gò Vấp, Hồ Chí Minh
- 2121: Cầu Giấy, Quan Hoa, Cầu Giấy, Hà Nội
- 2127: Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh
- 2345: 6, Yên Nghĩa, Hà Đông, Hà Nội
- 2399: Khu dân cư Hòn Rớ 1, Phước Đồng, Nha Trang, Khánh Hòa
- 2430: Đại La, Đồng Tâm, Hai Bà Trưng, Hà Nội
- 2448: 427, Liên Phương, Thường Tín, Hà Nội
- 2457: FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hành Sơn, Đà Nẵng
- 2460: Lê Hồng Phong, 1, Quận 10, Hồ Chí Minh
- 2537: Cầu Giấy, Quan Hoa, Cầu Giấy, Hà Nội
- 2538: Nguyễn Hữu Dật, Tây Thạnh, Tân Phú, Hồ Chí Minh
- 2558, 2578, 2580, 2598, 2634 3263, 3283, 3313, 3450,3661, 3740, 3804, 4036, 4067, 4094, 

In [209]:
nan_ward = cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['Tỉnh/Thành phố', 'Thành phố/Quận/Huyện/Thị xã','Xã/Phường/Thị trấn', 'short_address', 'description_x']]

In [210]:
nan_ward

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,short_address,description_x
179,Thành phố Hồ Chí Minh,Tân Bình,NaN,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...,"Nhà riêng 2 mặt tiền, cực chất ở Trường Chinh,..."
308,Thành phố Đà Nẵng,Liên Chiểu,NaN,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...,Chính chủ gửi bán nhà 3 tầng MT đường Nguyễn V...
373,Thành phố Hà Nội,Hai Bà Trưng,NaN,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội",Nhà riêng Bạch Mai dân xây khung cột bê tông c...
479,Thành phố Hà Nội,Long Biên,NaN,"Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội","Vị trí: Nguyễn Sơn, Long Biên ngõ thông, vài b..."
571,Thành phố Hồ Chí Minh,Bình Thạnh,NaN,"Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh","- Cần bán nhà riêng 3 tầng, 4PN + 5WC + bếp, p..."
...,...,...,...,...,...
26835,Thành phố Hà Nội,Long Biên,NaN,"Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",Cơ hội có 1 - 0 - 2 cho nhà đầu tư thông thái ...
26857,Thành phố Hà Nội,Long Biên,NaN,"Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",Cơ hội HIẾM CÓ sở hữu mảnh đất với vị trí kim ...
27065,Thành phố Hà Nội,Sóc Sơn,NaN,Phú thịnh Minh phú sóc sơn Hà Nội,Chính chủ em cần ra hàng lô đất.\nDT 100m².\nM...
27113,Tỉnh Bình Dương,Bàu Bàng,NaN,"Quốc Lộ 13, Lai Uyên, Bàu Bàng, Bình Dương","Đất nền tại Quốc Lộ 13, Lai Uyên, Bàu Bàng, Bì..."


In [211]:
import chardet

with open('Dữ liệu địa giới hành chính/streets_20250225_2.sql', 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [212]:
import sqlite3
import re
import os

def load_data(provinces_sql_path: str, districts_sql_path: str, wards_sql_path: str):
    try:
        conn = sqlite3.connect(":memory:")
        conn.execute("CREATE TABLE provinces (name TEXT, code TEXT, status TEXT);")
        conn.execute("CREATE TABLE districts (name TEXT, code TEXT, province_code TEXT, status TEXT);")
        conn.execute("CREATE TABLE wards (name TEXT, code TEXT, district_code TEXT, status TEXT);")
        # conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")

        with open(provinces_sql_path, "r", encoding="utf-8") as f:
            conn.executescript(f.read())

        with open(districts_sql_path, "r", encoding="utf-8") as f:
            dis_cleaned = f.read().replace("\\'", "''")
            conn.executescript(dis_cleaned)
        
        with open(wards_sql_path, "r", encoding="utf-8") as f:
            ward_cleaned = f.read().replace("\\'", "''")
            conn.executescript(ward_cleaned)

        # with open(streets_sql_path, "r", encoding="utf-8") as f:
        #     # street_cleaned = f.read().replace("'\\'", "''")
        #     street_cleaned = f.read().replace("'\\'", "''")
        #     street_cleaned = street_cleaned.replace("'\'", "''")
        #     street_cleaned = street_cleaned.replace("'", "''")
        #     conn.executescript(street_cleaned)

        provinces_df = pd.read_sql_query("SELECT * FROM provinces", conn)

        districts_df = pd.read_sql_query("""
            SELECT d.name AS district_name, p.name AS province_name
            FROM districts d
            JOIN provinces p ON d.province_code = p.code
            """, conn)
        
        wards_df = pd.read_sql_query("""
            SELECT w.name AS ward_name,
                   d.name AS district_name,
                   p.name AS province_name
            FROM wards w
            JOIN districts d ON w.district_code = d.code
            JOIN provinces p ON d.province_code = p.code
        """, conn)

        # # Join để street có thêm district + province
        # streets_df = pd.read_sql_query("""
        #         SELECT s.name AS street_name, s.code AS street_code, s.status AS street_status,
        #             d.name AS district_name, d.code AS district_code,
        #             p.name AS province_name, p.code AS province_code
        #         FROM streets s
        #         JOIN districts d ON s.district_code = d.code
        #         JOIN provinces p ON d.province_code = p.code
        #     """, conn)

    except (FileNotFoundError, sqlite3.Error) as e:
        print(f"Error: Could not load administrative data. Details: {e}")
        raise
    finally:
        if 'conn' in locals():
            conn.close()

    # Build reverse lookup maps
    reverse_province_map = {
        prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
        for prov in provinces_df['name'].unique()
    }
    reverse_province_map['Bà Rịa Vũng Tàu'] = 'Tỉnh Bà Rịa - Vũng Tàu'

    # reverse_district_map = {
    #     dis.replace("Thành phố ", "").replace("Thành Phố ", "").replace("Huyện ", "")
    #         .replace("Quận ", "").replace("Thị xã ", ""): dis
    #     for dis in districts_df['name'].unique()
    # }

    return [reverse_province_map, districts_df, wards_df]

def load_streets_into_dataframe(sql_file_path: str):
    """
    Reads a SQL script from a file, handles special characters, and loads the
    data into a pandas DataFrame via an in-memory SQLite database.
    This version gracefully handles and skips problematic SQL statements.

    Args:
        sql_file_path (str): The file path to the SQL script.

    Returns:
        pd.DataFrame: A DataFrame containing the street data, or an empty
                      DataFrame if an error occurs.
    """
    conn = None
    try:
        # Create an in-memory SQLite database
        conn = sqlite3.connect(":memory:")
        print("Connected to in-memory SQLite database.")

        # Create the table schema
        conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")
        print("Created 'streets' table.")

        # Check if the file exists
        if not os.path.exists(sql_file_path):
            print(f"Error: The file '{sql_file_path}' was not found.")
            return pd.DataFrame()

        print("Starting statement-by-statement execution...")
        
        # Read the SQL file content line by line to handle multi-line statements
        full_statement = ""
        with open(sql_file_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f, 1):
                full_statement += line.strip() + " "  # Add a space to prevent token merging
                
                # Check if the line ends with a semicolon (marking end of statement)
                if line.strip().endswith(';'):
                    try:
                        # Before execution, clean the statement
                        cleaned_statement = full_statement.replace("\\'", "''").replace("\\", "\\\\")
                        
                        conn.executescript(cleaned_statement)
                        full_statement = ""  # Reset for the next statement
                    except sqlite3.OperationalError as e:
                        print(f"--------------------------------------------------")
                        print(f"Skipping problematic statement ending on line {i}:")
                        print(f"SQL statement: {full_statement.strip()}")
                        print(f"Error details: {e}")
                        print(f"--------------------------------------------------")
                        full_statement = "" # Reset and continue to the next statement
                        continue # Continue the loop to the next line
        
        conn.commit()
        print("Successfully executed all valid SQL statements.")

        # Read the data from the database into a pandas DataFrame
        streets_df = pd.read_sql_query("SELECT * FROM streets", conn)
        print("Successfully loaded data into DataFrame.")
        
        return streets_df

    except sqlite3.OperationalError as e:
        print(f"An OperationalError occurred during SQL execution: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")
    
    # Return an empty DataFrame on error
    return pd.DataFrame()

province, district, ward = load_data("Dữ liệu địa giới hành chính/provinces_20250225_2.sql", 
                                     "Dữ liệu địa giới hành chính/districts_20250225_2.sql",
                                     "Dữ liệu địa giới hành chính/wards_20250225_2.sql")
street = load_streets_into_dataframe('Dữ liệu địa giới hành chính/streets_20250225_2.sql')

Connected to in-memory SQLite database.
Created 'streets' table.
Starting statement-by-statement execution...
--------------------------------------------------
Skipping problematic statement ending on line 23961:
SQL statement: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES ('\\', '48695925', '24120616', 'INACTIVE');
Error details: near "48695925": syntax error
--------------------------------------------------
--------------------------------------------------
Skipping problematic statement ending on line 23975:
SQL statement: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES (',.\\', '48697729', '24120616', 'INACTIVE');
Error details: near "48697729": syntax error
--------------------------------------------------
--------------------------------------------------
Skipping problematic statement ending on line 24037:
SQL statement: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES (',.\\', '48705771', '24122976',

In [213]:
district

,district_name,province_name
0,Thành phố Long Xuyên,Tỉnh An Giang
1,Thành phố Châu Đốc,Tỉnh An Giang
2,Thị xã Tân Châu,Tỉnh An Giang
3,Huyện An Phú,Tỉnh An Giang
4,Huyện Châu Phú,Tỉnh An Giang
...,...,...
702,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng
703,Thị xã Ngã Năm,Tỉnh Sóc Trăng
704,Huyện Thạnh Trị,Tỉnh Sóc Trăng
705,Thị xã Vĩnh Châu,Tỉnh Sóc Trăng


In [214]:
from unicodedata import normalize

reverse_district = {}

for province in district['province_name'].unique():
    reverse_district[province] = {}
    for district_name in district[district['province_name'] == province]['district_name'].unique():
        district_name_strip = normalize('NFKD', district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip())
        reverse_district[province][district_name_strip] = district_name

reverse_district

{'Tỉnh An Giang': {'Long Xuyên': 'Thành phố Long Xuyên',
  'Châu Đốc': 'Thành phố Châu Đốc',
  'Tân Châu': 'Thị xã Tân Châu',
  'An Phú': 'Huyện An Phú',
  'Châu Phú': 'Huyện Châu Phú',
  'Châu Thành': 'Huyện Châu Thành',
  'Chợ Mới': 'Huyện Chợ Mới',
  'Phú Tân': 'Huyện Phú Tân',
  'Thoại Sơn': 'Huyện Thoại Sơn',
  'Tịnh Biên': 'Thị xã Tịnh Biên',
  'Tri Tôn': 'Huyện Tri Tôn'},
 'Tỉnh Bắc Ninh': {'Yên Phong': 'Huyện Yên Phong',
  'Thuận Thành': 'Thị xã Thuận Thành',
  'Quế Võ': 'Thị xã Quế Võ',
  'Lương Tài': 'Huyện Lương Tài',
  'Gia Bình': 'Huyện Gia Bình',
  'Tiên Du': 'Huyện Tiên Du',
  'Từ Sơn': 'Thành phố Từ Sơn',
  'Bắc Ninh': 'Thành phố Bắc Ninh'},
 'Thành phố Hà Nội': {'Sơn Tây': 'Thị xã Sơn Tây',
  'Tây Hồ': 'Quận Tây Hồ',
  'Ba Vì': 'Huyện Ba Vì',
  'Chương Mỹ': 'Huyện Chương Mỹ',
  'Đan Phượng': 'Huyện Đan Phượng',
  'Đông Anh': 'Huyện Đông Anh',
  'Gia Lâm': 'Huyện Gia Lâm',
  'Hoài Đức': 'Huyện Hoài Đức',
  'Mê Lin

In [ ]:
from unicodedata import normalize

reverse_ward = {}

for district in ward['district_name'].unique():
    reverse_ward[district] = {}
    for ward_name in ward[ward['district_name'] == district]['ward_name'].unique():
        ward_name_strip = normalize('NFKD', ward_name.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
        # ward_name_strip = ward_name.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip()
        reverse_ward[district][ward_name_strip] = ward_name

reverse_ward

{'Thành phố Long Xuyên': {'Bình Khánh': 'Phường Bình Khánh',
  'Bình Đức': 'Phường Bình Đức',
  'Mỹ Bình': 'Phường Mỹ Bình',
  'Mỹ Hòa': 'Phường Mỹ Hòa',
  'Mỹ Long': 'Phường Mỹ Long',
  'Mỹ Phước': 'Phường Mỹ Phước',
  'Mỹ Quý': 'Phường Mỹ Quý',
  'Mỹ Thạnh': 'Phường Mỹ Thạnh',
  'Mỹ Thới': 'Phường Mỹ Thới',
  'Mỹ Xuyên': 'Phường Mỹ Xuyên',
  'Đông Xuyên': 'Phường Đông Xuyên',
  'Mỹ Hòa Hưng': 'Xã Mỹ Hòa Hưng',
  'Mỹ Khánh': 'Xã Mỹ Khánh'},
 'Thành phố Châu Đốc': {'Châu Phú A': 'Phường Châu Phú A',
  'Châu Phú B': 'Phường Châu Phú B',
  'Núi Sam': 'Phường Núi Sam',
  'Vĩnh Mỹ': 'Phường Vĩnh Mỹ',
  'Vĩnh Ngươn': 'Phường Vĩnh Ngươn',
  'Vĩnh Châu': 'Xã Vĩnh Châu',
  'Vĩnh Tế': 'Xã Vĩnh Tế'},
 'Thị xã Tân Châu': {'Long Châu': 'Phường Long Châu',
  'Long Hưng': 'Phường Long Hưng',
  'Long Phú': 'Phường Long Phú',
  'Long Sơn': 'Phường Long Sơn',
  'Long Thạnh': 'Phường Long Thạnh',
  'Châu Phong': 'Xã Châu Phong',
  'Long An': 'Xã Long An

In [224]:
cleaned['Thành phố/Quận/Huyện/Thị xã'].unique()

array(['Nha Trang', 'Nam Từ Liêm', 'Phú Nhuận', 'Cầu Giấy', 'Bình Tân',
       'Tân Bình', 'Nhà Bè', 'Hoài Đức', 'Quận 1', 'Quận 9', 'Bình Chánh',
       'Đống Đa', 'Gò Vấp', 'Thủ Đức', 'Hai Bà Trưng', 'Quận 10',
       'Bình Thạnh', 'Hoàng Mai', 'Hà Đông', 'Long Biên', 'Quận 3',
       'Tây Hồ', 'Hoàn Kiếm', 'Hóc Môn', 'Quận 12', 'Quận 7', 'Quận 2',
       'Thanh Xuân', 'Quận 8', 'Tân Phú', 'Hiệp Hòa', 'Ba Đình',
       'Quận 11', 'Đông Anh', 'Củ Chi', 'Lê Chân', 'Đồ Sơn', 'Quận 6',
       'Bắc Từ Liêm', 'Quận 5', 'Hòa Vang', 'Quận 4', 'Đức Hòa',
       'Vĩnh Yên', 'Liên Chiểu', 'Gia Lâm', 'An Dương', 'Hồng Bàng',
       'Hải An', 'Thanh Trì', 'Biên Hòa', 'Cẩm Lệ', 'Sơn Trà',
       'Ngũ Hành Sơn', 'Ngô Quyền', 'Đà Lạt', 'Nhơn Trạch', 'Ia Grai',
       'Thanh Hóa', 'Tân An', 'Ninh Kiều', 'Thanh Khê', 'Ninh Bình',
       'Lương Sơn', 'Quốc Oai', 'Hải Châu', 'Dĩ An', 'Ninh Hòa',
       'Bến Cát', 'Thủ Dầu Một', 'Huế', 'Bình Thủy', 'Hưng Yên',
       'Đông Hòa', 'Sa Đéc', 'Văn Giang', 'C

In [225]:
yo = 'hello   '
yo2 = yo.replace(' ', 'n')
print(yo)
print(yo2)

hello   
hellonnn


In [223]:
def clean_district(row):
    if isinstance(row['Thành phố/Quận/Huyện/Thị xã'], str):
        return row['Thành phố/Quận/Huyện/Thị xã']
    short_add_list = row['short_address'].split(',')
    if len(short_add_list) >= 3:
        district_true = normalize('NFKD', short_add_list[-2].strip())
        province = row['Tỉnh/Thành phố']
        if province in reverse_district.keys():
            # Add this line to see what the keys actually are
            print(f"Keys in reverse_district[{province}] are: {list(reverse_district[province].keys())}")
            print(f"The district_true value is: '{district_true}'")
            if reverse_district[province][district_true]:
                print(f'Return value: {reverse_district[province][district_true]}')
                return reverse_district[province][district_true]
        return None

def clean_ward(row):
    if isinstance(row['Xã/Phường/Thị trấn'], str):
        return row['Xã/Phường/Thị trấn']
    short_add_list = row['short_address'].split(',')
    if len(short_add_list) >= 4:
        print(short_add_list)
        ward_true = normalize('NFKD', short_add_list[-3].strip())
        # ward_true = short_add_list[-3].strip()
        quanhuyen = row['Thành phố/Quận/Huyện/Thị xã']
        print(quanhuyen)
        if quanhuyen in reverse_ward.keys():
            # Add this line to see what the keys actually are
            print(f"Keys in reverse_ward[{quanhuyen}] are: {list(reverse_ward[quanhuyen].keys())}")
            print(f"The ward_true value is: '{ward_true}'")
            try:
                print(f'Return value: {reverse_ward[quanhuyen][ward_true]}')
                return reverse_ward[quanhuyen][ward_true]
            except:
                if len(ward_true) == 1:
                    ward_true = ward_true.zfill(2)
                    print(reverse_ward[quanhuyen][ward_true])
                    return reverse_ward[quanhuyen][ward_true]
                else:
                    return 'Phường' + ward_true
        return None

cleaned['Xã/Phường/Thị trấn'] = cleaned.apply(clean_ward, axis = 1)
print(cleaned['Xã/Phường/Thị trấn'].isna().sum())

['Bạch Mai', ' Bạch Mai', ' Hai Bà Trưng', ' Hà Nội']
Hai Bà Trưng
['Nguyễn Sơn', ' Bồ Đề', ' Long Biên', ' Hà Nội']
Long Biên
['Đặng Thùy Trâm', ' 13', ' Bình Thạnh', ' Hồ Chí Minh']
Bình Thạnh
['Thạch Cầu', ' Long Biên', ' Long Biên', ' Hà Nội']
Long Biên
['Liên Mạc', ' Thượng Cát', ' Bắc Từ Liêm', ' Hà Nội']
Bắc Từ Liêm
['Trịnh Văn Bô', ' Xuân Phương', ' Nam Từ Liêm', ' Hà Nội']
Nam Từ Liêm
['Đông Tác', ' Kim Liên', ' Đống Đa', ' Hà Nội']
Đống Đa
['Tên Lửa', ' Bình Trị Đông B', ' Bình Tân', ' Hồ Chí Minh']
Bình Tân
['Hương lộ 2', ' Bình Trị Đông', ' Bình Tân', ' Hồ Chí Minh']
Bình Tân
['Nguyễn Trãi', ' Nguyễn Trãi', ' Thượng Đình', ' Thanh Xuân', ' Hà Nội']
Thanh Xuân
['Tên Lửa', ' Bình Trị Đông B', ' Bình Tân', ' Hồ Chí Minh']
Bình Tân
['Trịnh Như Khuê', ' Bình Chánh', ' Bình Chánh', ' Hồ Chí Minh']
Bình Chánh
['Nguyễn Văn Bứa', ' Xuân Thới Sơn', ' Hóc Môn', ' Hồ Chí Minh']
Hóc Môn
['Xuân Đỉnh', ' Xuân Đỉnh', ' Bắc Từ Liêm', ' Hà Nội']
Bắc Từ Liêm
['Vạn Kiếp', ' 3', ' Bình Thạnh',

In [195]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['short_address']]

,short_address
179,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Trường Chinh
308,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiểu Đà Nẵng
373,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội"
479,"Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội"
571,"Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh"
626,"Thạch Cầu, Long Biên, Long Biên, Hà Nội"
714,"Liên Mạc, Thượng Cát, Bắc Từ Liêm, Hà Nội"
783,"Trịnh Văn Bô, Xuân Phương, Nam Từ Liêm, Hà Nội"
839,"Đông Tác, Kim Liên, Đống Đa, Hà Nội"
840,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh"


In [22]:
def clean_ward(row):
    if isinstance(row['Xã/Phường/Thị trấn'], str):
        return row['Xã/Phường/Thị trấn']
    short_add_list = row['short_address'].split(',')
    if len(short_add_list) >= 4:
        ward_true = short_add_list[-3].strip()
        quanhuyen = row['Thành phố/Quận/Huyện/Thị xã']
        ward_list = ward[ward['district_name'] == quanhuyen]['ward_name'].tolist()
        ward_list = [i.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '') for i in ward_list]
        for i in ward_list:
            if ward_true == i:
                return i
        # return match_ward(norm_text(ward_true), ward_list)
    return None

cleaned['Xã/Phường/Thị trấn'] = cleaned.apply(clean_ward, axis = 1)
print(cleaned['Xã/Phường/Thị trấn'].isna().sum())
cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['short_address']]

58


,short_address
176,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...
305,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...
368,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội"
828,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh"
1042,"Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh"
1169,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh"
2052,"Vạn Kiếp, 3, Bình Thạnh, Hồ Chí Minh"
2430,"Đại La, Đồng Tâm, Hai Bà Trưng, Hà Nội"
2457,"FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hà..."
2460,"Lê Hồng Phong, 1, Quận 10, Hồ Chí Minh"


In [238]:
from rapidfuzz import process, fuzz
import unicodedata

def match_ward(name, choices, score_cutoff=60):
    """
    name: tên phường từ short_address
    choices: list ward chuẩn
    score_cutoff: ngưỡng tối thiểu để chấp nhận
    """
    if not isinstance(name, str):
        return None
    
    # Lấy best match
    match = process.extractOne(
        name, 
        choices, 
        scorer=fuzz.ratio, 
        score_cutoff=score_cutoff
    )
    
    if match:
        print(f'{name}:{match[0]}')
        return match[0]   # match[0] là tên ward tốt nhất
    return None

def norm_text(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFD', text).strip()

def clean_ward(row):
    if isinstance(row['Xã/Phường/Thị trấn'], str):
        return row['Xã/Phường/Thị trấn']
    short_add_list = row['short_address'].split(',')
    if len(short_add_list) >= 4:
        ward_true = short_add_list[-3].strip()
        quanhuyen = row['Thành phố/Quận/Huyện/Thị xã']
        print(quanhuyen)
        ward_list = ward[ward['district_name'] == quanhuyen]['ward_name'].tolist()
        print(f'{quanhuyen}: {ward_list}')
        for i in ward_list:
            if norm_text(ward_true) in norm_text(i):
                return i
        return match_ward(norm_text(ward_true), ward_list)
    return None

# Áp dụng cho dataframe
# cleaned['matched_ward'] = cleaned['short_address'].apply(
#     lambda x: match_ward(str(x).split(',')[-3].strip(), ward_list)
# )
cleaned['Xã/Phường/Thị trấn'] = cleaned.apply(clean_ward, axis=1)

Huyện Bình Tân
Huyện Bình Tân: ['Thị trấn Tân Quới', 'Xã Mỹ Thuận', 'Xã Nguyễn Văn Thảnh', 'Xã Thành Lợi', 'Xã Thành Trung', 'Xã Tân An Thạnh', 'Xã Tân Bình', 'Xã Tân Hưng', 'Xã Tân Lược', 'Xã Tân Thành']
Huyện Bình Tân
Huyện Bình Tân: ['Thị trấn Tân Quới', 'Xã Mỹ Thuận', 'Xã Nguyễn Văn Thảnh', 'Xã Thành Lợi', 'Xã Thành Trung', 'Xã Tân An Thạnh', 'Xã Tân Bình', 'Xã Tân Hưng', 'Xã Tân Lược', 'Xã Tân Thành']
Huyện Bình Tân
Huyện Bình Tân: ['Thị trấn Tân Quới', 'Xã Mỹ Thuận', 'Xã Nguyễn Văn Thảnh', 'Xã Thành Lợi', 'Xã Thành Trung', 'Xã Tân An Thạnh', 'Xã Tân Bình', 'Xã Tân Hưng', 'Xã Tân Lược', 'Xã Tân Thành']
Quận Ngũ Hành Sơn
Quận Ngũ Hành Sơn: ['Phường Hoà Hải', 'Phường Hòa Quý', 'Phường Khuê Mỹ', 'Phường Mỹ An']
Huyện Tân Phú
Huyện Tân Phú: ['Thị trấn Tân Phú', 'Xã Nam Cát Tiên', 'Xã Núi Tượng', 'Xã Phú An', 'Xã Phú Bình', 'Xã Phú Lâ

In [236]:
pd.set_option('display.max_colwidth', None)
print(cleaned['Xã/Phường/Thị trấn'].isna().sum())

cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['short_address']]

40


,short_address
176,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Trường Chinh
305,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiểu Đà Nẵng
828,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh"
1042,"Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh"
1169,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh"
2457,"FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hành Sơn, Đà Nẵng"
2538,"Nguyễn Hữu Dật, Tây Thạnh, Tân Phú, Hồ Chí Minh"
4036,"Lê Tấn Bê, An Lạc, Bình Tân, Hồ Chí Minh"
4897,"Bằng Liệt, Hoàng Liệt, Hoàng Mai, Hà Nội"
5794,"Khu biệt thự Sunflower, Lê Lai, Hòa Phú, Thủ Dầu Một, Bình Dương"


In [ ]:
import unicodedata
import re

nan_ward['short_add_list'] = nan_ward['short_address'].apply(lambda x: x.split(','))

# def norm_text(text):
#     if isinstance(text, str):
#         # 1. Normalize to composed form (fix precomposed/decomposed)
#         text = unicodedata.normalize("NFC", text)
#         # 2. Replace non-breaking and zero-width spaces with normal space
#         text = text.replace("\u00A0", " ").replace("\u200B", "")
#         # 3. Lowercase (optional, for case-insensitive matching)
#         text = text.lower()
#         # 4. Collapse multiple spaces/commas
#         text = re.sub(r"\s*,\s*", ", ", text)   # tidy commas
#         text = re.sub(r"\s+", " ", text)        # collapse spaces
#         # 5. Strip
#         return text.strip()
#     return text

def norm_text(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFD', text).strip()

def clean_ward(row):
    if isinstance(row['Xã/Phường/Thị trấn'], str):
        return row['Xã/Phường/Thị trấn']
    short_add_list = row['short_address'].split(',')
    quanhuyen = row['Thành phố/Quận/Huyện/Thị xã']
    ward_names = ward[ward['district_name'] == quanhuyen]['ward_name'].unique()
    if len(short_add_list) >= 4:
        for i in ward_names:
            if norm_text(short_add_list[-3]) in norm_text(i):
                return unicodedata.normalize('NFD', i).strip()
    return None
        
cleaned['Xã/Phường/Thị trấn'] = cleaned.apply(clean_ward, axis=1)
cleaned['Xã/Phường/Thị trấn'].isna().sum()

np.int64(42)

In [ ]:
cleaned['Xã/Phường/Thị trấn'].unique()

array(['Phường Phương Sơn', 'Phường Tây Mỗ', 'Phường 5',
       'Phường Quan Hoa', 'Phường Tân Tạo', 'Phường 13', 'Xã Phước Lộc',
       'Xã An Khánh', 'Phường Bến Thành', 'Phường Phú Hữu', 'Xã Tân Kiên',
       'Phường Đại Mỗ', 'Phường Láng Hạ', 'Phường Ngọc Hiệp', 'Phường 15',
       'Phường Láng Hạ', 'Phường 9', 'Phường Hiệp Bình Chánh',
       'Phường Thanh Lương', 'Phường 7', 'Phường Nghĩa Đô', 'Phường 14',
       'Phường Hoàng Văn Thụ', 'Phường 1', 'Thị trấn Tân Túc',
       'Phường Hà Cầu', 'Phường 3', 'Phường Vĩnh Hải',
       'Phường Trung Liệt', 'Phường Thạch Bàn', 'Phường Phương Canh',
       'Phường 11', 'Phường 8', 'Phường Bình Trị Đông', 'Phường 25',
       'Phường Xuân La', 'Phường 19', 'Phường 17', 'Phường 6',
       'Phường Trần Hưng Đạo', 'Phường 12', 'Phường Nghĩa Tân',
       'Phường Ngọc Thụy', 'Phường Ô Chợ Dừa', 'Xã Hiệp Phước',
       'Phường Hiệp Thành', 'Phường Tăng Nhơn Phú A', 'Phường Phú Mỹ',
       'Phường Vĩnh Hòa', 'Phường Bình Trưng Đông',
      

### Đang bị vấn đề ở đây, không biết làm sao mà xét short_add_list[-3] in ward_name không đúng

In [ ]:
pd.set_option('display.max_colwidth', None)
cleaned[cleaned['Xã/Phường/Thị trấn'].isna()][['short_address', 'Xã/Phường/Thị trấn']]

,short_address,Xã/Phường/Thị trấn
176,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Trường Chinh,None
305,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiểu Đà Nẵng,None
828,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh",None
1042,"Hương lộ 2, Bình Trị Đông, Bình Tân, Hồ Chí Minh",None
1169,"Tên Lửa, Bình Trị Đông B, Bình Tân, Hồ Chí Minh",None
2457,"FPT City Đà Nẵng, Võ Chí Công, Hòa Hải, Ngũ Hành Sơn, Đà Nẵng",None
2538,"Nguyễn Hữu Dật, Tây Thạnh, Tân Phú, Hồ Chí Minh",None
4036,"Lê Tấn Bê, An Lạc, Bình Tân, Hồ Chí Minh",None
4640,"Đường Mậu Lương, TDP12 Phường Kiến Hưng, Hà Đông, Hà Nội",None
4897,"Bằng Liệt, Hoàng Liệt, Hoàng Mai, Hà Nội",None
